In [10]:
import unicodedata
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver

import numpy as np

In [11]:
from googletrans import Translator, constants
from pprint import pprint
import pandas as pd
def translate(key):
    translator = Translator()
    if(key is None):
        return None
    
    if isinstance(key, list):
        return [translator.translate(i).text for i in key]
    else :
        translation = translator.translate(key)
        return translation.text


In [12]:
main_url='https://oda.com/de/products/'
response = requests.get(main_url)
soup = BeautifulSoup(response.content, 'lxml')
all_cats = soup.find_all(class_='k-grid-span-12 lg:k-grid-span-6 xl:k-grid-span-4 k-column')
all_cats = [i.find(class_='k-link AppLink_LinkContainer__bPYAe k-link--text k-link--text-decoration-none AppLink_AppLink__Uu3_c')['href'] for i in all_cats]
all_cats = ['https://oda.com/'+i for i in all_cats]

In [13]:
def get_all_sub_cats(i):
    response = requests.get(i)
    soup = BeautifulSoup(response.content, 'lxml')
    cat_name = soup.find(class_='k-text-style k-text-style--display-l k-text-color--deprecated_purple').text.strip()
    sub_cats = soup.find_all(class_= 'k-chip k-chip--medium AppLink_AppLink__Uu3_c')
    sub_cats = ['https://oda.com/'+i['href'] for i in sub_cats]
    sub_cats = [s for s in sub_cats if 'filter' not in s]

    return translate(cat_name),sub_cats

In [14]:
def get_all_product_links(sub_cat):
    response = requests.get(sub_cat)
    soup = BeautifulSoup(response.content, 'lxml')
    sub_cat_name = soup.find(class_='k-text-style k-text-style--display-l k-text-color--deprecated_purple').text
    all_products = soup.find_all(class_='k-grid-span-6 md:k-grid-span-4 xl:k-grid-span-2 k-column')
    all_products = ['https://oda.com/' + i.find('a')['href'] for i in all_products]
    return translate(sub_cat_name),all_products
    

In [15]:
def get_product(url):
    
    translated  = url.replace('oda.com' , 'oda-com.translate.goog')
    translated = translated +"?_x_tr_sl=de&_x_tr_tl=en&_x_tr_hl=en&_x_tr_pto=wapp&_x_tr_hist=true"
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(options=options)
    driver.get(translated)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    product = {}

    p_name = soup.find(class_='k-text-style k-text-style--display-xs k-text-color--deprecated_purple').text.strip()
    try:
        p_price = soup.find(class_='k-text-style k-text-style--headline-l k-text--weight-medium ProductPrice_price__ze6_3').text.strip()
    except:
        try:
            p_price = soup.find(class_='k-text-style k-text-style--headline-l k-text--weight-medium ProductPrice_price__ze6_3 ProductPrice_price-discount__UjZGB').text.strip()
        except:
            p_price="0"
    p_price = p_price.replace('\xa0','')
    p_info=""
    try:
        p_info = p_info + soup.find(class_='k-text-style k-text-style--body-m k-text--pre-wrap').text.strip()
    except:
        pass
    try:
        p_info = p_info + " , " + soup.find(class_='k-text-style k-text-style--body-m k-mb-6 lg:k-mb-7').text.strip()
    except:
        pass
    ans = soup.find_all(class_='k-grid-span-12 lg:k-grid-span-8 k-column')
    ans = [i.text for i in ans]
    nutrion = soup.find_all(class_='k-grid-span-12 lg:k-grid-span-4 k-column')
    nutrion = [i.text for i in nutrion]
    
    product['PROD_NAME'] = p_name
    product['PROD_PRICE'] = p_price
    product['PROD_DESCRIPTION'] = translate(p_info)
    product['PROD_URL'] = url
    product['PROD_IMAGE_URL'] = soup.find(class_='k-aspect-ratio k-aspect-ratio--4-3').find('img')['src']


    
    i = len(nutrion)-1 
    j = len(ans)-1
    while(j>=0 and i>=0):
        product[nutrion[i]] = ans[j]
        i-=1
        j-=1
    driver.quit()    
    return product
    

In [ ]:
for j in range(0, len(all_cats)):
    ps = []
    cat_name , sub_cats_links = get_all_sub_cats(all_cats[j])
    print("CAT = " ,cat_name)
    for j in sub_cats_links:
        sub_cat_name , all_products_links = get_all_product_links(j)
        print(sub_cat_name , " - " , len(all_products_links))
        for m,k in enumerate(all_products_links):
            print(m)
            try:
                product = get_product(k)
                product['PROD_CATEGORY'] = cat_name
                product['PROD_SUB_CATEGORY'] = sub_cat_name
                product['STORE_NAME'] = 'oda'
                ps.append(product)
            except:
                print("ERROR" , k)
    df = pd.DataFrame(ps)
    df.to_csv(fr"E:\Omdena\Berlin-Chapter\Task1\oda_scrap\{cat_name}.csv")
    print("*******" )
        

CAT =  Meat, sausage & fish
specialties  -  6
0
1
2
3
4
5
Meat & meat alternatives  -  144
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
Sausage & cold cuts  -  303
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
1